In [3]:
#loading libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
#overriding default setting of pandas
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gc

In [4]:
#load all data files
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
prior_order = pd.read_csv('order_products__prior.csv')
train_order = pd.read_csv('order_products__train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

#imputation 
orders.fillna(value = 0, inplace = True)

Merge all the files together using the order_id as a key

In [5]:
prior_orders_data = prior_order.sort_values(by=['order_id'])
prior_orders_data = pd.merge(left = prior_orders_data, right = products,
                             left_on='product_id', right_on='product_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_orders_data = pd.merge(left = prior_orders_data, right = aisles,
                             left_on='aisle_id', right_on='aisle_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_orders_data = pd.merge(left = prior_orders_data, right = departments,
                             left_on='department_id', right_on='department_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_orders_data = pd.merge(left = prior_orders_data, right = orders,
                             left_on='order_id', right_on='order_id').sort_values(by=['order_id']).reset_index(drop=True)

col_order = ['user_id',
 'order_id',
 'product_id',
  'aisle_id',
 'department_id',
 'add_to_cart_order',
 'reordered',
 'product_name',
 'aisle',
 'department',
 'eval_set',
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'days_since_prior_order']

prior_orders_data = prior_orders_data[col_order]
print("Shape of prior_orders_data :", prior_orders_data.shape)
prior_orders_data.head()

prior_orders_data = pd.concat([prior_order, train_order]).sort_values(by=['order_id'])
prior_orders_data = pd.merge(left = prior_orders_data, right = products,
                            left_on='product_id', right_on='product_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_orders_data = pd.merge(left = prior_orders_data, right = aisles,
                            left_on='aisle_id', right_on='aisle_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_orders_data = pd.merge(left = prior_orders_data, right = departments,
                        left_on='department_id', right_on='department_id').sort_values(by=['order_id']).reset_index(drop=True)
prior_orders_data = pd.merge(left = prior_orders_data, right = orders,
                             left_on='order_id', right_on='order_id').sort_values(by=['order_id']).reset_index(drop=True)

prior_orders_data.head()

prior_orders_data.groupby(["user_id","product_id"]).size().shape[0]/prior_orders_data.shape[0]

prior_orders_data.head()

Shape of prior_orders_data : (32434489, 15)


,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,112108,train,4,4,10,9.0
1,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0
2,1,13176,6,0,Bag of Organic Bananas,24,4,fresh fruits,produce,112108,train,4,4,10,9.0
3,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods,112108,train,4,4,10,9.0
4,1,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0


# Preprocessing Strategy: 

**Generate Training Data (using prior order data and train order data)**
- Build features on prior order data
- Order from train order data for every user will be used to label the dependent variable i.e. reordered.
            For every orders in prior_orders_data, get last order_id for every user, say take n-1 order for feature engineering.
        nth order of every user will used to label the dependent variable i.e. reordered.

        example : --

        let, user A have 90 orders in prior_orders_data.

          - build features using 89 orders.
          - based on these we will label the data with reordered(0/1) if any of the products he brought in 89 orders
          appeared in his 90th order
- Predict the product reorder probability using trained model to give accuracy.
- Pick top probable products whose probability of reordering was high

**Generate Test Data ( using order.eval =='test')**
- Add features built on training data , based on orders and users
- For every order and product predict if it is reordered(0/1)
- Then we will pick top probable products whose probability of reordering

# Product Based Features

Features:

    feat_1 : product_reorder_rate       : How Frequently the product was reordered ?
    feat_2 : avg_pos_incart             : Average position of product in the cart ?
    
    next 3 values are calculated based product being (these are the ailes or departments to which the product belongs) 
        - organic, 
        - isYogurt - aisle
        - produce  - department
        - isFrozen  - department 
        - isdairy  - department
        - isbreakfast  - department 
        - issnack  - department
        - isbeverage  - department
        
    Because there are a lot of departments/aisles, these values are then reduced to 3 columns using Non-Negative Matrix Factorization, to reduce sparsity. NMF is used to reduce the dimensions. 
        
    feat_3 : p_reduced_feat_1             : column 1 from NMF output
    feat_4 : p_reduced_feat_2             : column 2 from NMF output
    feat_5 : p_reduced_feat_3             : column 3 from NMF output
    
    feat_6 : aisle_reorder_rate           : How frequently a product is reordered from the aisle to which this product belongs
    feat_7 : department_reorder_rate      : How frequently a product is reordered from the department to which this product belongs

In [6]:

def generate_product_features(prior_data = None):
    #create an empty dataframe
    product_features = pd.DataFrame(columns=['product_id'])

    #add product_name
    product_features['product_id'] = prior_data['product_id'].sort_values().unique()

    #get reorder_rate for each product
    #reorder_rate = reorders / total orders
    df = pd.DataFrame({'reorder_rate': prior_data.groupby(['product_id','reordered'])['reordered'].\
                                                       count().groupby(level=0).\
                                                       apply(lambda x: x / float(x.sum()))}).reset_index()

    #get data of reordered products only
    new_df = df[df['reordered']==1]
    new_df['reorder_rate'] = new_df['reorder_rate'] * new_df['reordered']

    #handling for products which were never reordered, hence reorder_rate = 0.0
    new_df_1 = df[(df['reordered']==0) & (df['reorder_rate']==float(1.0))]
    new_df_1['reorder_rate'] = new_df_1['reorder_rate'] * new_df_1['reordered']
    new_df = new_df.append(new_df_1)

    #drop other columns of the new_df and sort values by product name to align with product features dataframe
    new_df.drop('reordered', axis = 1, inplace = True)
    new_df.sort_values(by='product_id', inplace =  True)   
    new_df = new_df.reset_index(drop = True)

    #add to feat_1 of product_features dataframe
    product_features['product_reorder_rate'] = new_df['reorder_rate']

    #get mean position of product in the cart, sort by product_name and add to feat_2 of product_features
    mean_position = prior_data.groupby('product_id')['add_to_cart_order'].mean().reset_index(name = 'mean_position')
    mean_position.sort_values(by = 'product_id', inplace = True)
    product_features['avg_pos_incart'] = mean_position['mean_position']



    #generate boolean values if product belongs to below categories
    products['organic'] = products['product_name'].apply(lambda x: 'organic' in x.lower()).astype(int)
    products['isYogurt'] = products['aisle_id'].apply(lambda x: x==120).astype(int)
    products['isProduce'] = products['department_id'].apply(lambda x: x==4).astype(int)
    products['isFrozen'] = products['department_id'].apply(lambda x: x==1).astype(int)
    products['isdairy'] = products['department_id'].apply(lambda x: x==16).astype(int)
    products['isbreakfast'] = products['department_id'].apply(lambda x: x==14).astype(int)
    products['issnack'] = products['department_id'].apply(lambda x: x==19).astype(int)
    products['isbeverage'] = products['department_id'].apply(lambda x: x==7).astype(int)

    new_product_feat = products[['organic', 'isYogurt', 'isProduce', 'isFrozen', 'isdairy', 'isbreakfast', 'issnack', 'isbeverage']]
    
    #reduce sparsity using NMF
    
#I found this and copied it from: https://www.kaggle.com/themissingsock/matrix-decomposition-with-buyer-data
    from sklearn.decomposition import NMF
    from sklearn.preprocessing import normalize

    nmf = NMF(n_components = 3)
    model = nmf.fit(new_product_feat)
    W = model.transform(new_product_feat)
    prod_data = pd.DataFrame(normalize(W))

    prod_data.columns = ['p_reduced_feat_1', 'p_reduced_feat_2','p_reduced_feat_3']
    products.drop(['organic', 'isYogurt', 'isProduce', 'isFrozen', 'isdairy', 'isbreakfast', 'issnack', 'isbeverage'], axis = 1, inplace =True)

    product_features['p_reduced_feat_1'] = prod_data['p_reduced_feat_1']
    product_features['p_reduced_feat_2'] = prod_data['p_reduced_feat_2']
    product_features['p_reduced_feat_3'] = prod_data['p_reduced_feat_3']
    

    #Get aisle reorder rate
    df = prior_data.groupby(['aisle']).size().reset_index(name='order_count')
    aisle_reorder_rate = prior_data[prior_data['reordered']==1].groupby(['aisle']).size().reset_index(name='reorder_rate')
    df['aisle_reorder_rate'] = aisle_reorder_rate['reorder_rate']/df['order_count']
    df.drop(['order_count'], axis = 1, inplace = True)
    new_df = pd.merge(prior_data, df, on = 'aisle')
    
    
    #Get dept reorder rate
    df = prior_data.groupby(['department']).size().reset_index(name='order_count')
    aisle_reorder_rate = prior_data[prior_data['reordered']==1].groupby(['aisle']).size().reset_index(name='reorder_rate')
    df['dept_reorder_rate'] = aisle_reorder_rate['reorder_rate']/df['order_count']
    df.drop(['order_count'], axis = 1, inplace = True)
    new_df = pd.merge(new_df, df, on = 'department')
    
    new_df = new_df[['product_id','aisle_id','department_id','aisle_reorder_rate','dept_reorder_rate']]
    new_df.drop_duplicates(keep='first', inplace = True)

    #merge dept_reorder_rate and aisle_reorder_rate to existing product features
    product_features = pd.merge(product_features, new_df , on='product_id', how = 'inner')
    
    del df, new_df, new_df_1, new_product_feat, model, prod_data
    return product_features

In [7]:
product_features = generate_product_features(prior_data = prior_orders_data )
product_features.to_csv("product_features_v6.csv",index=False)

<ipython-input-6-5767f667a2aa>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['reorder_rate'] = new_df['reorder_rate'] * new_df['reordered']
<ipython-input-6-5767f667a2aa>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_1['reorder_rate'] = new_df_1['reorder_rate'] * new_df_1['reordered']


In [8]:
product_features

,product_id,product_reorder_rate,avg_pos_incart,p_reduced_feat_1,p_reduced_feat_2,p_reduced_feat_3,aisle_id,department_id,aisle_reorder_rate,dept_reorder_rate
0,1,0.614627,5.845954,1.0,0.0,0.0,61,19,0.549341,0.043527
1,2,0.138298,10.138298,0.0,0.0,0.0,104,13,0.152933,0.032061
2,3,0.738516,6.374558,0.0,0.0,1.0,94,7,0.527619,0.000936
3,4,0.458689,9.472934,0.0,1.0,0.0,38,1,0.558656,0.006066
4,5,0.625000,6.375000,0.0,0.0,0.0,5,13,0.281312,0.032061
...,...,...,...,...,...,...,...,...,...,...
49680,49684,0.111111,4.333333,0.0,1.0,0.0,124,5,0.572844,0.040805
49681,49685,0.122449,9.571429,0.0,0.0,1.0,42,1,0.544497,0.006066
49682,49686,0.700787,7.433071,0.0,1.0,0.0,112,3,0.670552,0.003811
49683,49687,0.428571,7.071429,0.0,0.0,0.0,41,8,0.622251,1.748985


# User Features 

Now, I'll generate user based features and return it as dataframe object.
    merge it with the product features afterwards
    
   **Features:** 
    
    feat_1 : user_reorder_rate              : Average reorder rate on orders placed?
    
           **Get mean reorder rate for every user** 

             Process :
              1. Count reorders for every user_id using reordered feature of prior_train_orders
              2. Perform Groupby on this output using user_id and get prob of reorder == 0 and reorder == 1
              3. Seperate reorder == 0 and reorder == 1 using pivot and fill 0 , where reorder==1 is NA ( indicating no reorders)
              4. Add reorder == 1 column to feat_1 
   
    feat_2 : user_unique_products           : Distinct Products ordered ?
    feat_3 : user_total_products            : Total products ordered ?
    feat_4 : user_avg_cart_size             : mean Products per order ? = average cart size ?
    feat_5 : user_avg_days_between_orders   : Average days between orders ?
    feat_6 : user_reordered_products_ratio  : user product reorder ratio

In [9]:
def generate_user_features(prior_data = None):
    
    #create an empty dataframe
    user_features = pd.DataFrame(columns=['user_id'])
    
    #Add user_id
    user_features['user_id'] = prior_data['user_id'].sort_values().unique()
    
    user_reorder_rate = prior_data.groupby(["user_id","reordered"])['reordered'].count().groupby(level = 0).apply(lambda x: x / float(x.sum())).reset_index(name='reorder_rate')
    user_reorder_rate = user_reorder_rate.pivot(index ='user_id', columns ='reordered', values =['reorder_rate']) 
    user_reorder_rate = pd.DataFrame(user_reorder_rate.to_records())
    user_reorder_rate.columns = ['user_id','0', '1']
    user_reorder_rate.set_index("user_id", inplace = True)
    user_reorder_rate.fillna(0, inplace = True)
    user_reorder_rate.reset_index(inplace = True)
    user_features['user_reorder_rate'] = user_reorder_rate['1']
    
    #Get count of all unique products for every user
    user_features['user_unique_products'] = prior_data.groupby(["user_id"])['product_name'].nunique().reset_index(name = 'unique')['unique']
    
    #Get count of all products ordered by user
    user_features['user_total_products'] = prior_data.groupby(["user_id"])['product_name'].size().reset_index(name = 'count')['count']
    
    #Get mean products per user = Average cart size of user
    df = prior_data.groupby(["user_id","order_id"])['add_to_cart_order'].count().reset_index(name='cart_size')\
                                                                .groupby('user_id')['cart_size'].mean().reset_index()
    user_features['user_avg_cart_size'] = df['cart_size']
    
    #Get average days between 2 orders for every user
    df = prior_data.groupby(["user_id","order_id"])['days_since_prior_order'].max().reset_index(name='mean_days_between_orders')\
                                                                .groupby('user_id')['mean_days_between_orders'].mean().reset_index()
    user_features['user_avg_days_between_orders'] = df['mean_days_between_orders']
    
    
    #get user product reorder ratio 
    # number of unique products reordered / number of unique products ordered
    df['user_id'] = prior_data['user_id'].sort_values().unique()
    df['user_unique_products'] = prior_data.groupby(["user_id"])['product_name'].nunique().reset_index(name = 'unique')['unique']
    df['user_reordered_products'] = prior_data[prior_data['reordered']==1].groupby(["user_id"])['product_name'].nunique().reset_index(name = 'reordered_unique')['reordered_unique']
    df.fillna(0, inplace = True)
    user_features['user_reordered_products_ratio'] = df['user_reordered_products'] / df['user_unique_products']
    
    del df
    return user_features

In [10]:
user_features = generate_user_features(prior_data = prior_orders_data)
user_features.to_csv("user_features_v6.csv",index=False)

# User Product Interaction

Generate User-Product interaction features and return it as dataframe object, and merge this later too... 
    
  **Features**
    
    feat_1 : u_p_order_rate             :  How frequently user ordered the product ?
    feat_2 : u_p_reorder_rate           :  How frequently user reordered the product ?
    feat_3 : u_p_avg_position           :  Average position of product in the cart on orders placed by user ?
    feat_4 : u_p_orders_since_last      :  Number of orders placed since the product was last ordered ?
    feat_5 : max_streak                 :  Number of orders where user continuously brought a product without missing... counting the streak...
        
           I used stackoverflow to do the max streak calculation

In [11]:
def max_streak(row):
    _max = 0
    _sum = 0
    for i in row:
        if i==1:
            _sum += 1
        else:
            if _sum > _max:
                _max = _sum
            _sum = 0 
    return _max

In [12]:
def generate_user_product_features(prior_data = None):

    #create an empty dataframe
    user_product_features = pd.DataFrame(columns=['user_id','product_id'])
    
    #get unique user-product pairs ( total data is reduced by 60 %)
    #prior_train_orders.groupby(["user_id","product_id"]).size().shape[0]/prior_train_orders.shape[0]  - 0.409
    #add user and product to dataframe
    u_p = prior_data.groupby(["user_id","product_id"]).size().reset_index()
    user_product_features["user_id"] = u_p["user_id"]
    user_product_features["product_id"] = u_p["product_id"]
    
    #How frequently user ordered the product ?
    # #times user ordered the product/ #times user placed an order
    df = prior_data.groupby(["user_id","product_id"])["reordered"].size()
    df = df/prior_data.groupby(["user_id"]).size()
    df = df.reset_index(name = 'order_rate')
    df.fillna(0. , inplace = True)
    user_product_features["u_p_order_rate"] = df["order_rate"]
    
    #How frequently user reordered the product ?
    # #times user reordered the product/ #times user ordered the product
    df = prior_data[prior_data["reordered"]==1].groupby(["user_id","product_id"])["reordered"].size()
    df = df/prior_data.groupby(["user_id","product_id"]).size()
    df = df.reset_index(name = 'reorder_rate')
    df.fillna(0. , inplace = True)
    user_product_features["u_p_reorder_rate"] = df["reorder_rate"]
    
    #Average position of product in the cart on orders placed by user ?
    
    df = prior_data.groupby(["user_id","product_id"])['add_to_cart_order'].mean().reset_index(name = 'mean_position')
    user_product_features['u_p_avg_position'] = df['mean_position']

    
    #Number of orders placed since the product was last ordered ?
    # Get last order_number placed by user , subtract with last order_number with the product in cart 
    
    df = prior_data.groupby(["user_id","product_id"])['order_number'].max().reset_index()
    df_2 = prior_data.groupby(["user_id"])['order_number'].max().reset_index()
    new_df = pd.merge(df, df_2,  how='outer', left_on=['user_id'], right_on = ['user_id'])        
    new_df['order_diff'] = new_df['order_number_y'] - new_df['order_number_x']
    user_product_features['u_p_orders_since_last'] = new_df['order_diff']
    
    #max_streak
    df = prior_data.groupby(["user_id","product_id"])['reordered'].apply(list).reset_index(name = 'max_streak')
    df['max_streak'] = df['max_streak'].apply(max_streak)
    user_product_features = pd.merge(user_product_features, df, on= ["user_id","product_id"])
    #user_features["max_streak"] = df['reorder_summary'].apply(max_streak) 
    
    
    del df, new_df, df_2
    return user_product_features

In [13]:
user_product_features = generate_user_product_features(prior_data = prior_orders_data)
user_product_features.to_csv("user_product_features_prior_data_v6.csv",index=False)

# More features for Product Time, inspired from EDA: 
     **features**
- product_time: How frequently product was reordered on any given hour ?
- product_day: How frequently product was reordered on any given day ?
- product_days_since_prior: How frequently a product was reordered  given that difference between 2 orders containing product in days ?
- user_days_since_prior: How frequently user reordered any product given difference between 2 orders in days ?
- u_p_days_since_prior: How frequently user reordered a particular product given difference between 2 orders in days ?

In [14]:
def product_time(prior_data = None):
    df = prior_data.groupby(['product_id','order_hour_of_day'])["reordered"].size()
    df = df/prior_data.groupby(["product_id"]).size()
    df = df.reset_index(name = 'hour_reorder_rate')
    return df
#construct dataframe
hour_reorder_rate = product_time(prior_orders_data)
#save these files
hour_reorder_rate.to_csv("hour_reorder_rate.csv", index = False)

def product_day(prior_data = None):
    df = prior_data.groupby(['product_id','order_dow'])["reordered"].size()
    df = df/prior_data.groupby(["product_id"]).size()
    df = df.reset_index(name = 'day_reorder_rate')
    return df
day_reorder_rate = product_day(prior_orders_data)
day_reorder_rate.to_csv("day_reorder_rate.csv", index = False)


def product_days_since_prior(prior_data = None):
    df = prior_data.groupby(['product_id','days_since_prior_order'])["reordered"].size()
    df = df/prior_data.groupby(["product_id"]).size()
    df = df.reset_index(name = 'p_days_since_prior_order_reorder_rate')
    return df
p_days_since_prior_order_reorder_rate = product_days_since_prior(prior_orders_data)
p_days_since_prior_order_reorder_rate.to_csv("p_days_since_prior_order_reorder_rate.csv", index = False)



def user_days_since_prior(prior_data = None):
    df = prior_data.groupby(['user_id','days_since_prior_order'])["reordered"].size()
    df = df/prior_data.groupby(["user_id"]).size()
    df = df.reset_index(name = 'u_days_since_prior_order_reorder_rate')
    return df
u_days_since_prior_order_reorder_rate = user_days_since_prior(prior_orders_data)
u_days_since_prior_order_reorder_rate.to_csv("u_days_since_prior_order_reorder_rate.csv", index = False)


def u_p_days_since_prior(prior_data = None):
    df = prior_data.groupby(["user_id","product_id","days_since_prior_order"])["reordered"].size()
    df = df/prior_data.groupby(["user_id","product_id"]).size()
    df = df.reset_index(name = 'days_since_prior_reorder_rate')
    return df
days_since_prior_reorder_rate = u_p_days_since_prior(prior_orders_data)
days_since_prior_reorder_rate.to_csv("days_since_prior_reorder_rate.csv", index = False)

# Call and Merge the data and Prepare the Training Set

In [15]:
product_features = pd.read_csv("product_features_v6.csv")
user_features = pd.read_csv("user_features_v6.csv")
user_product_features = pd.read_csv("user_product_features_prior_data_v6.csv")

In [16]:
merged_df = pd.merge(user_product_features, user_features,  how='outer', left_on=['user_id'], right_on = ['user_id'])
merged_df = pd.merge(merged_df, product_features,  how='outer', left_on=['product_id'], right_on = ['product_id'])

In [17]:
order_details_train = orders[orders['eval_set'] == 'train']
order_details_train, train_order

(         order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  days_since_prior_order
 10        1187899        1    train            11          4                  8                    14.0
 25        1492625        2    train            15          1                 11                    30.0
 49        2196797        5    train             5          0                 11                     6.0
 74         525192        7    train            21          2                 11                     6.0
 78         880375        8    train             4          1                 14                    10.0
 ...           ...      ...      ...           ...        ...                ...                     ...
 3420838   2585586   206199    train            20          2                 16                    30.0
 3420862    943915   206200    train            24          6                 19                     6.0
 3420924   2371631   206203    train             6     

# Merge all the others product time features: 

In [18]:
train_order_data = train_order.sort_values(by=['order_id'])

train_order_data = pd.merge(left = train_order_data, right = products,
                             left_on='product_id', right_on='product_id').sort_values(by=['order_id']).reset_index(drop=True)
train_order_data = pd.merge(left = train_order_data, right = aisles,
                             left_on='aisle_id', right_on='aisle_id').sort_values(by=['order_id']).reset_index(drop=True)
train_order_data = pd.merge(left = train_order_data, right = departments,
                             left_on='department_id', right_on='department_id').sort_values(by=['order_id']).reset_index(drop=True)
train_order_data = pd.merge(left = train_order_data, right = orders,
                             left_on='order_id', right_on='order_id').sort_values(by=['order_id']).reset_index(drop=True)

col_order = ['user_id',
 'order_id',
 'product_id',
  'aisle_id',
 'department_id',
 'add_to_cart_order',
 'reordered',
 'product_name',
 'aisle',
 'department',
 'eval_set',
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'days_since_prior_order']

train_order_data = train_order_data[col_order]
print("Shape of train_order_data :", train_order_data.shape)

Shape of train_order_data : (1384617, 15)


In [19]:
upd_train_orders = train_order_data[['user_id','order_id','product_id','reordered']]
last_orders = upd_train_orders.groupby(['user_id'])['order_id'].max().reset_index(name = 'new_order_id')
order_details = train_order_data[['order_id','order_dow','order_hour_of_day','days_since_prior_order']]
order_details = order_details.drop_duplicates()

#merge latest orders (prior_last_orders) with above user and product feat
train_orders_merged_df = pd.merge(merged_df, upd_train_orders,  how='left', left_on=['user_id','product_id'], right_on = ['user_id','product_id'])
train_orders_merged_df = pd.merge(train_orders_merged_df, last_orders, on = 'user_id')
train_orders_merged_df.drop("order_id", axis = 1, inplace = True)
train_orders_merged_df.rename(columns = {'new_order_id':'order_id'}, inplace = True) 
train_orders_merged_df = pd.merge(train_orders_merged_df, order_details, on = 'order_id')
train_orders_merged_df[['reordered']]= train_orders_merged_df[['reordered']].fillna(value=0.0)

#merge with all the prodcuct time features
train_orders_merged_df = pd.merge(train_orders_merged_df, hour_reorder_rate, on=['product_id','order_hour_of_day'], how = 'left')
train_orders_merged_df[['hour_reorder_rate']]= train_orders_merged_df[['hour_reorder_rate']].fillna(value=0.0)

train_orders_merged_df = pd.merge(train_orders_merged_df, day_reorder_rate, on=['product_id','order_dow'], how = 'left')
train_orders_merged_df[['day_reorder_rate']]= train_orders_merged_df[['day_reorder_rate']].fillna(value=0.0)

train_orders_merged_df = pd.merge(train_orders_merged_df, p_days_since_prior_order_reorder_rate, on=['product_id','days_since_prior_order'], how = 'left')
train_orders_merged_df[['p_days_since_prior_order_reorder_rate']]= train_orders_merged_df[['p_days_since_prior_order_reorder_rate']].fillna(value=0.0)

train_orders_merged_df = pd.merge(train_orders_merged_df, u_days_since_prior_order_reorder_rate, on=['user_id','days_since_prior_order'], how = 'left')
train_orders_merged_df[['u_days_since_prior_order_reorder_rate']]= train_orders_merged_df[['u_days_since_prior_order_reorder_rate']].fillna(value=0.0)

train_orders_merged_df = pd.merge(train_orders_merged_df, days_since_prior_reorder_rate, on=["user_id","product_id",'days_since_prior_order'], how = 'left')
train_orders_merged_df[['days_since_prior_reorder_rate']]= train_orders_merged_df[['days_since_prior_reorder_rate']].fillna(value=0.0)


train_orders_merged_df = train_orders_merged_df[['user_id', 'product_id', 'u_p_order_rate', 'u_p_reorder_rate', 'u_p_avg_position', 'u_p_orders_since_last', 'max_streak', 'user_reorder_rate', 'user_unique_products', 'user_total_products', 'user_avg_cart_size', 'user_avg_days_between_orders', 'user_reordered_products_ratio', 'product_reorder_rate', 'avg_pos_incart', 'p_reduced_feat_1', 'p_reduced_feat_2', 'p_reduced_feat_3', 'aisle_id', 'department_id', 'aisle_reorder_rate', 'dept_reorder_rate', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'hour_reorder_rate', 'day_reorder_rate', 'p_days_since_prior_order_reorder_rate', 'u_days_since_prior_order_reorder_rate', 'days_since_prior_reorder_rate', 'order_id','reordered']]

train_orders_merged_df

,user_id,product_id,u_p_order_rate,u_p_reorder_rate,u_p_avg_position,u_p_orders_since_last,max_streak,user_reorder_rate,user_unique_products,user_total_products,user_avg_cart_size,user_avg_days_between_orders,user_reordered_products_ratio,product_reorder_rate,avg_pos_incart,p_reduced_feat_1,p_reduced_feat_2,p_reduced_feat_3,aisle_id,department_id,aisle_reorder_rate,dept_reorder_rate,order_dow,order_hour_of_day,days_since_prior_order,hour_reorder_rate,day_reorder_rate,p_days_since_prior_order_reorder_rate,u_days_since_prior_order_reorder_rate,days_since_prior_reorder_rate,order_id,reordered
0,1,196,0.157143,0.909091,1.363636,0,7,0.728571,19,70,6.363636,17.272727,0.631579,0.777843,3.732399,0.0,0.0,1.0,77,7,0.639301,0.000936,4,8,14.0,0.059092,0.152448,0.038340,0.242857,0.181818,1187899,1.0
1,1,10258,0.142857,0.900000,3.600000,0,6,0.728571,19,70,6.363636,17.272727,0.631579,0.715610,4.268293,0.0,0.0,0.0,117,19,0.519358,0.043527,4,8,14.0,0.061951,0.140488,0.040976,0.242857,0.200000,1187899,1.0
2,1,10326,0.014286,0.000000,5.000000,6,0,0.728571,19,70,6.363636,17.272727,0.631579,0.653439,4.227846,0.0,0.0,0.0,24,4,0.718823,0.005192,4,8,14.0,0.067765,0.124264,0.035648,0.242857,0.000000,1187899,0.0
3,1,12427,0.142857,0.900000,3.300000,1,6,0.728571,19,70,6.363636,17.272727,0.631579,0.740182,4.781245,0.0,0.0,0.0,23,19,0.592129,0.043527,4,8,14.0,0.066149,0.130208,0.042108,0.242857,0.100000,1187899,0.0
4,1,13032,0.057143,0.750000,6.500000,0,2,0.728571,19,70,6.363636,17.272727,0.631579,0.661117,5.632172,0.0,0.0,0.0,121,14,0.572233,0.315318,4,8,14.0,0.063781,0.142418,0.033811,0.242857,0.250000,1187899,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9030449,120516,40072,0.333333,0.666667,1.000000,0,2,0.666667,3,9,1.125000,14.250000,13.666667,0.448276,6.172414,0.0,0.0,1.0,88,13,0.487940,0.032061,2,9,14.0,0.172414,0.137931,0.103448,0.222222,0.333333,3264957,1.0
9030450,164455,16838,0.857143,0.833333,1.000000,0,1,0.714286,2,7,1.166667,25.000000,37.000000,0.450980,3.516340,0.0,0.0,0.0,70,11,0.445901,0.326152,1,15,30.0,0.071895,0.124183,0.300654,0.857143,0.833333,2432273,1.0
9030451,164455,28952,0.142857,0.000000,2.000000,0,0,0.714286,2,7,1.166667,25.000000,37.000000,0.255556,3.677778,1.0,0.0,0.0,47,11,0.312176,0.326152,1,15,30.0,0.066667,0.155556,0.288889,0.857143,1.000000,2432273,0.0
9030452,178074,19318,1.000000,0.923077,1.000000,0,2,0.923077,1,13,1.000000,5.153846,55.000000,0.759563,9.278689,0.0,0.0,0.0,17,13,0.305445,0.032061,5,11,7.0,0.081967,0.196721,0.071038,0.153846,0.153846,2250575,1.0


In [20]:
train_orders_merged_df

,user_id,product_id,u_p_order_rate,u_p_reorder_rate,u_p_avg_position,u_p_orders_since_last,max_streak,user_reorder_rate,user_unique_products,user_total_products,user_avg_cart_size,user_avg_days_between_orders,user_reordered_products_ratio,product_reorder_rate,avg_pos_incart,p_reduced_feat_1,p_reduced_feat_2,p_reduced_feat_3,aisle_id,department_id,aisle_reorder_rate,dept_reorder_rate,order_dow,order_hour_of_day,days_since_prior_order,hour_reorder_rate,day_reorder_rate,p_days_since_prior_order_reorder_rate,u_days_since_prior_order_reorder_rate,days_since_prior_reorder_rate,order_id,reordered
0,1,196,0.157143,0.909091,1.363636,0,7,0.728571,19,70,6.363636,17.272727,0.631579,0.777843,3.732399,0.0,0.0,1.0,77,7,0.639301,0.000936,4,8,14.0,0.059092,0.152448,0.038340,0.242857,0.181818,1187899,1.0
1,1,10258,0.142857,0.900000,3.600000,0,6,0.728571,19,70,6.363636,17.272727,0.631579,0.715610,4.268293,0.0,0.0,0.0,117,19,0.519358,0.043527,4,8,14.0,0.061951,0.140488,0.040976,0.242857,0.200000,1187899,1.0
2,1,10326,0.014286,0.000000,5.000000,6,0,0.728571,19,70,6.363636,17.272727,0.631579,0.653439,4.227846,0.0,0.0,0.0,24,4,0.718823,0.005192,4,8,14.0,0.067765,0.124264,0.035648,0.242857,0.000000,1187899,0.0
3,1,12427,0.142857,0.900000,3.300000,1,6,0.728571,19,70,6.363636,17.272727,0.631579,0.740182,4.781245,0.0,0.0,0.0,23,19,0.592129,0.043527,4,8,14.0,0.066149,0.130208,0.042108,0.242857,0.100000,1187899,0.0
4,1,13032,0.057143,0.750000,6.500000,0,2,0.728571,19,70,6.363636,17.272727,0.631579,0.661117,5.632172,0.0,0.0,0.0,121,14,0.572233,0.315318,4,8,14.0,0.063781,0.142418,0.033811,0.242857,0.250000,1187899,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9030449,120516,40072,0.333333,0.666667,1.000000,0,2,0.666667,3,9,1.125000,14.250000,13.666667,0.448276,6.172414,0.0,0.0,1.0,88,13,0.487940,0.032061,2,9,14.0,0.172414,0.137931,0.103448,0.222222,0.333333,3264957,1.0
9030450,164455,16838,0.857143,0.833333,1.000000,0,1,0.714286,2,7,1.166667,25.000000,37.000000,0.450980,3.516340,0.0,0.0,0.0,70,11,0.445901,0.326152,1,15,30.0,0.071895,0.124183,0.300654,0.857143,0.833333,2432273,1.0
9030451,164455,28952,0.142857,0.000000,2.000000,0,0,0.714286,2,7,1.166667,25.000000,37.000000,0.255556,3.677778,1.0,0.0,0.0,47,11,0.312176,0.326152,1,15,30.0,0.066667,0.155556,0.288889,0.857143,1.000000,2432273,0.0
9030452,178074,19318,1.000000,0.923077,1.000000,0,2,0.923077,1,13,1.000000,5.153846,55.000000,0.759563,9.278689,0.0,0.0,0.0,17,13,0.305445,0.032061,5,11,7.0,0.081967,0.196721,0.071038,0.153846,0.153846,2250575,1.0


## Converting time to a categorical variable (3 sections)

In [21]:
#https://learnersdictionary.com/qa/parts-of-the-day-early-morning-late-morning-etc

# For ease, change the time to a categorical variable

def hour_tocategorical(time):
    if time > 5 and time < 12:
            return 0
    elif time > 12 and time < 17:
        return 1
    elif time > 17 and time < 21:
        return 2
    else:
        return 3

In [22]:
#convert order_hour_of_day to categorical feature 
train_orders_merged_df['order_hour_of_day'] = train_orders_merged_df['order_hour_of_day'].apply(hour_tocategorical)
train_orders_merged_df.reordered.value_counts()
train_orders_merged_df

,user_id,product_id,u_p_order_rate,u_p_reorder_rate,u_p_avg_position,u_p_orders_since_last,max_streak,user_reorder_rate,user_unique_products,user_total_products,user_avg_cart_size,user_avg_days_between_orders,user_reordered_products_ratio,product_reorder_rate,avg_pos_incart,p_reduced_feat_1,p_reduced_feat_2,p_reduced_feat_3,aisle_id,department_id,aisle_reorder_rate,dept_reorder_rate,order_dow,order_hour_of_day,days_since_prior_order,hour_reorder_rate,day_reorder_rate,p_days_since_prior_order_reorder_rate,u_days_since_prior_order_reorder_rate,days_since_prior_reorder_rate,order_id,reordered
0,1,196,0.157143,0.909091,1.363636,0,7,0.728571,19,70,6.363636,17.272727,0.631579,0.777843,3.732399,0.0,0.0,1.0,77,7,0.639301,0.000936,4,0,14.0,0.059092,0.152448,0.038340,0.242857,0.181818,1187899,1.0
1,1,10258,0.142857,0.900000,3.600000,0,6,0.728571,19,70,6.363636,17.272727,0.631579,0.715610,4.268293,0.0,0.0,0.0,117,19,0.519358,0.043527,4,0,14.0,0.061951,0.140488,0.040976,0.242857,0.200000,1187899,1.0
2,1,10326,0.014286,0.000000,5.000000,6,0,0.728571,19,70,6.363636,17.272727,0.631579,0.653439,4.227846,0.0,0.0,0.0,24,4,0.718823,0.005192,4,0,14.0,0.067765,0.124264,0.035648,0.242857,0.000000,1187899,0.0
3,1,12427,0.142857,0.900000,3.300000,1,6,0.728571,19,70,6.363636,17.272727,0.631579,0.740182,4.781245,0.0,0.0,0.0,23,19,0.592129,0.043527,4,0,14.0,0.066149,0.130208,0.042108,0.242857,0.100000,1187899,0.0
4,1,13032,0.057143,0.750000,6.500000,0,2,0.728571,19,70,6.363636,17.272727,0.631579,0.661117,5.632172,0.0,0.0,0.0,121,14,0.572233,0.315318,4,0,14.0,0.063781,0.142418,0.033811,0.242857,0.250000,1187899,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9030449,120516,40072,0.333333,0.666667,1.000000,0,2,0.666667,3,9,1.125000,14.250000,13.666667,0.448276,6.172414,0.0,0.0,1.0,88,13,0.487940,0.032061,2,0,14.0,0.172414,0.137931,0.103448,0.222222,0.333333,3264957,1.0
9030450,164455,16838,0.857143,0.833333,1.000000,0,1,0.714286,2,7,1.166667,25.000000,37.000000,0.450980,3.516340,0.0,0.0,0.0,70,11,0.445901,0.326152,1,1,30.0,0.071895,0.124183,0.300654,0.857143,0.833333,2432273,1.0
9030451,164455,28952,0.142857,0.000000,2.000000,0,0,0.714286,2,7,1.166667,25.000000,37.000000,0.255556,3.677778,1.0,0.0,0.0,47,11,0.312176,0.326152,1,1,30.0,0.066667,0.155556,0.288889,0.857143,1.000000,2432273,0.0
9030452,178074,19318,1.000000,0.923077,1.000000,0,2,0.923077,1,13,1.000000,5.153846,55.000000,0.759563,9.278689,0.0,0.0,0.0,17,13,0.305445,0.032061,5,0,7.0,0.081967,0.196721,0.071038,0.153846,0.153846,2250575,1.0


In [23]:
#check correlation between variables
train_orders_merged_df[train_orders_merged_df.columns[:]].corr()['reordered'][:]

user_id                                 -0.000561
product_id                               0.002784
u_p_order_rate                           0.332684
u_p_reorder_rate                         0.418406
u_p_avg_position                        -0.049759
u_p_orders_since_last                   -0.228218
max_streak                               0.272472
user_reorder_rate                        0.043667
user_unique_products                    -0.083579
user_total_products                     -0.043050
user_avg_cart_size                       0.059473
user_avg_days_between_orders             0.051378
user_reordered_products_ratio            0.047842
product_reorder_rate                     0.164459
avg_pos_incart                          -0.125572
p_reduced_feat_1                        -0.006936
p_reduced_feat_2                        -0.001246
p_reduced_feat_3                         0.014745
aisle_id                                 0.002038
department_id                           -0.032566


In [24]:
from sys import getsizeof
print('Dataframe size: %2.2f GB'%(getsizeof(train_orders_merged_df)/(1024.0**3))) # change it into GB format

Dataframe size: 2.22 GB


In [25]:
train_orders_merged_df.isnull().sum()

user_id                                  0
product_id                               0
u_p_order_rate                           0
u_p_reorder_rate                         0
u_p_avg_position                         0
u_p_orders_since_last                    0
max_streak                               0
user_reorder_rate                        0
user_unique_products                     0
user_total_products                      0
user_avg_cart_size                       0
user_avg_days_between_orders             0
user_reordered_products_ratio            0
product_reorder_rate                     0
avg_pos_incart                           0
p_reduced_feat_1                         0
p_reduced_feat_2                         0
p_reduced_feat_3                         0
aisle_id                                 0
department_id                            0
aisle_reorder_rate                       0
dept_reorder_rate                        0
order_dow                                0
order_hour_

In [26]:
train_orders_merged_df.to_csv("train_orders_merged_df_v6.csv",index=False)

# Prepare the Testing Data

In [27]:
#get test order details
order_details_test = orders[orders['eval_set'] == 'test']
order_details_test.drop(['eval_set'], axis = 1, inplace = True)

# merge on user_product features, to get features on this order
test_orders_merge_df = pd.merge(order_details_test, user_product_features, on = ['user_id'], how = 'outer')
test_orders_merge_df.dropna(inplace =True)
test_orders_merge_df = pd.merge(test_orders_merge_df, user_features, on = ['user_id'])
test_orders_merge_df = pd.merge(test_orders_merge_df, product_features, on = ['product_id'])

/Users/amenti_kenea/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [28]:
order_details_test

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,13,5,15,11.0
44,329954,4,6,3,12,30.0
53,1528013,6,4,3,16,22.0
96,1376945,11,8,6,11,8.0
102,1356845,12,6,1,20,30.0
...,...,...,...,...,...,...
3420918,2728930,206202,23,2,17,6.0
3420929,350108,206204,5,4,14,14.0
3421001,1043943,206206,68,0,20,0.0
3421018,2821651,206207,17,2,13,14.0


In [29]:
# merge on user_product features, to get features on this order

test_orders_merge_df = pd.merge(order_details_test, user_product_features, on = ['user_id'], how = 'outer')
test_orders_merge_df.dropna(inplace =True)
test_orders_merge_df = pd.merge(test_orders_merge_df, user_features, on = ['user_id'])
test_orders_merge_df = pd.merge(test_orders_merge_df, product_features, on = ['product_id'])

In [30]:
#merge with misc features
test_orders_merge_df = pd.merge(test_orders_merge_df, hour_reorder_rate, on=['product_id','order_hour_of_day'], how = 'left')
test_orders_merge_df[['hour_reorder_rate']]= test_orders_merge_df[['hour_reorder_rate']].fillna(value=0.0)

test_orders_merge_df = pd.merge(test_orders_merge_df, day_reorder_rate, on=['product_id','order_dow'], how = 'left')
test_orders_merge_df[['day_reorder_rate']]= test_orders_merge_df[['day_reorder_rate']].fillna(value=0.0)

test_orders_merge_df = pd.merge(test_orders_merge_df, p_days_since_prior_order_reorder_rate, on=['product_id','days_since_prior_order'], how = 'left')
test_orders_merge_df[['p_days_since_prior_order_reorder_rate']]= test_orders_merge_df[['p_days_since_prior_order_reorder_rate']].fillna(value=0.0)

test_orders_merge_df = pd.merge(test_orders_merge_df, u_days_since_prior_order_reorder_rate, on=['user_id','days_since_prior_order'], how = 'left')
test_orders_merge_df[['u_days_since_prior_order_reorder_rate']]= test_orders_merge_df[['u_days_since_prior_order_reorder_rate']].fillna(value=0.0)

test_orders_merge_df = pd.merge(test_orders_merge_df, days_since_prior_reorder_rate, on=["user_id","product_id",'days_since_prior_order'], how = 'left')
test_orders_merge_df[['days_since_prior_reorder_rate']]= test_orders_merge_df[['days_since_prior_reorder_rate']].fillna(value=0.0)

In [31]:
test_orders_merge_df['order_hour_of_day'] = test_orders_merge_df['order_hour_of_day'].apply(hour_tocategorical)
test_orders_merge_df = test_orders_merge_df[['user_id', 'product_id', 'u_p_order_rate', 'u_p_reorder_rate', 'u_p_avg_position', 'u_p_orders_since_last', 'max_streak', 'user_reorder_rate', 'user_unique_products', 'user_total_products', 'user_avg_cart_size', 'user_avg_days_between_orders', 'user_reordered_products_ratio', 'product_reorder_rate', 'avg_pos_incart', 'p_reduced_feat_1', 'p_reduced_feat_2', 'p_reduced_feat_3', 'aisle_id', 'department_id', 'aisle_reorder_rate', 'dept_reorder_rate', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'hour_reorder_rate', 'day_reorder_rate', 'p_days_since_prior_order_reorder_rate', 'u_days_since_prior_order_reorder_rate', 'days_since_prior_reorder_rate', 'order_id']]
test_orders_merge_df

,user_id,product_id,u_p_order_rate,u_p_reorder_rate,u_p_avg_position,u_p_orders_since_last,max_streak,user_reorder_rate,user_unique_products,user_total_products,user_avg_cart_size,user_avg_days_between_orders,user_reordered_products_ratio,product_reorder_rate,avg_pos_incart,p_reduced_feat_1,p_reduced_feat_2,p_reduced_feat_3,aisle_id,department_id,aisle_reorder_rate,dept_reorder_rate,order_dow,order_hour_of_day,days_since_prior_order,hour_reorder_rate,day_reorder_rate,p_days_since_prior_order_reorder_rate,u_days_since_prior_order_reorder_rate,days_since_prior_reorder_rate,order_id
0,3,248,0.011364,0.000000,3.000000,10,0,0.625000,33,88,7.333333,11.083333,0.575758,0.401366,10.637178,1.000000,0.000000,0.000000,117,19,0.519358,0.043527,5.0,1,11.0,0.088316,0.123824,0.022003,0.056818,0.0,2774568.0
1,12,248,0.013514,0.000000,22.000000,0,0,0.175676,61,74,14.800000,20.000000,0.147541,0.401366,10.637178,1.000000,0.000000,0.000000,117,19,0.519358,0.043527,1.0,2,30.0,0.036419,0.172838,0.102124,0.527027,1.0,1356845.0
2,418,248,0.005988,0.000000,3.000000,3,0,0.359281,107,167,20.875000,20.625000,0.130841,0.401366,10.637178,1.000000,0.000000,0.000000,117,19,0.519358,0.043527,6.0,3,13.0,0.051593,0.145068,0.025948,0.137725,0.0,3073553.0
3,503,248,0.002994,0.000000,13.000000,22,0,0.613772,129,334,9.277778,9.777778,0.054264,0.401366,10.637178,1.000000,0.000000,0.000000,117,19,0.519358,0.043527,1.0,1,8.0,0.075873,0.172838,0.061760,0.176647,1.0,1490499.0
4,720,248,0.009709,0.666667,4.333333,2,1,0.618123,118,309,13.434783,13.782609,0.093220,0.401366,10.637178,1.000000,0.000000,0.000000,117,19,0.519358,0.043527,1.0,1,19.0,0.081032,0.172838,0.009712,0.000000,0.0,391588.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4833287,205877,27049,0.001684,0.000000,32.000000,5,0,0.501684,296,594,13.500000,7.795455,0.000000,0.363636,10.090909,0.000000,0.000000,0.000000,40,8,0.566198,1.748985,3.0,0,21.0,0.090909,0.181818,0.000000,0.000000,0.0,913357.0
4833288,205880,25635,0.014286,0.000000,1.000000,10,0,0.242857,53,70,5.384615,3.230769,0.000000,0.666667,4.333333,0.000000,0.000000,0.000000,33,6,0.398439,0.603735,3.0,3,1.0,0.111111,0.000000,0.000000,0.000000,0.0,2268059.0
4833289,205893,11873,0.002786,0.000000,34.000000,7,0,0.543175,164,359,22.437500,17.125000,0.000000,0.166667,13.833333,0.002308,0.999987,0.004505,80,11,0.238818,0.326152,0.0,0,21.0,0.000000,0.166667,0.000000,0.000000,0.0,2072901.0
4833290,206131,12653,0.013245,0.500000,7.000000,0,0,0.549669,68,151,12.583333,17.166667,0.000000,0.250000,10.750000,0.000000,1.000000,0.000000,37,1,0.491640,0.006066,1.0,1,13.0,0.250000,0.000000,0.000000,0.000000,0.0,2964171.0


In [32]:
from sys import getsizeof
print('Dataframe size: %2.2f GB'%(getsizeof(test_orders_merge_df)/(1024.0**3)))
test_orders_merge_df.to_csv("test_orders_merge_df_v6.csv",index=False)

Dataframe size: 1.15 GB


In [33]:
df = pd.read_csv("train_orders_merged_df_v6.csv")
df.head()

,user_id,product_id,u_p_order_rate,u_p_reorder_rate,u_p_avg_position,u_p_orders_since_last,max_streak,user_reorder_rate,user_unique_products,user_total_products,user_avg_cart_size,user_avg_days_between_orders,user_reordered_products_ratio,product_reorder_rate,avg_pos_incart,p_reduced_feat_1,p_reduced_feat_2,p_reduced_feat_3,aisle_id,department_id,aisle_reorder_rate,dept_reorder_rate,order_dow,order_hour_of_day,days_since_prior_order,hour_reorder_rate,day_reorder_rate,p_days_since_prior_order_reorder_rate,u_days_since_prior_order_reorder_rate,days_since_prior_reorder_rate,order_id,reordered
0,1,196,0.157143,0.909091,1.363636,0,7,0.728571,19,70,6.363636,17.272727,0.631579,0.777843,3.732399,0.0,0.0,1.0,77,7,0.639301,0.000936,4,0,14.0,0.059092,0.152448,0.038340,0.242857,0.181818,1187899,1.0
1,1,10258,0.142857,0.900000,3.600000,0,6,0.728571,19,70,6.363636,17.272727,0.631579,0.715610,4.268293,0.0,0.0,0.0,117,19,0.519358,0.043527,4,0,14.0,0.061951,0.140488,0.040976,0.242857,0.200000,1187899,1.0
2,1,10326,0.014286,0.000000,5.000000,6,0,0.728571,19,70,6.363636,17.272727,0.631579,0.653439,4.227846,0.0,0.0,0.0,24,4,0.718823,0.005192,4,0,14.0,0.067765,0.124264,0.035648,0.242857,0.000000,1187899,0.0
3,1,12427,0.142857,0.900000,3.300000,1,6,0.728571,19,70,6.363636,17.272727,0.631579,0.740182,4.781245,0.0,0.0,0.0,23,19,0.592129,0.043527,4,0,14.0,0.066149,0.130208,0.042108,0.242857,0.100000,1187899,0.0
4,1,13032,0.057143,0.750000,6.500000,0,2,0.728571,19,70,6.363636,17.272727,0.631579,0.661117,5.632172,0.0,0.0,0.0,121,14,0.572233,0.315318,4,0,14.0,0.063781,0.142418,0.033811,0.242857,0.250000,1187899,1.0


In [34]:
# End